In [1]:
import pandas as pd
import numpy as np
from malnis import show
import malnis
from sklearn.metrics import average_precision_score, roc_auc_score
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
import torch

/home/jarobyte/envs/malnis/lib/python3.8/site-packages/huggingface_hub/snapshot_download.py:6: FutureWarning: snapshot_download.py has been made private and will no longer be available from version 0.11. Please use `from huggingface_hub import snapshot_download` to import the only public function in this module. Other members of the file may be changed without a deprecation notice.
  warnings.warn(


In [2]:
data = pd.read_pickle("quotes.pkl")
show(data, 30)

(23, 3)


,text,query,relevance
0,"[29, R425–R473 Forum Using Soundscapes to Asse...",Hydrothermal vents are unique deep-sea structu...,"[0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0]"
1,"[Therefore, using daily SEL to describe sounds...",Hydrothermal vents are unique deep-sea structu...,"[1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, ..."
2,[TECHNOLOGY AND CODE published: 19 August 2021...,Hydrothermal vents are unique deep-sea structu...,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,[ORIGINAL RESEARCH published: 05 August 2021 d...,Hydrothermal vents are unique deep-sea structu...,"[0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, ..."
4,[A brief overview of current approaches for un...,Hydrothermal vents are unique deep-sea structu...,"[0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 0, ..."
5,[Due to its tremendous advantages this technol...,Cloud computing provides a shared pool of comp...,"[0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0]"
6,[Interpretable & Explorable Approximations of ...,Prognostic modelling using machine learning te...,"[1, 0, 1, 0, 1, 0]"
7,"[Introduction Over the past decade, there has ...",Prognostic modelling using machine learning te...,"[1, 1, 1, 0, 1, 1, 0, 0]"
9,"[To gain insight into the black-box model, we ...",Prognostic modelling using machine learning te...,"[1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1]"
11,"[Next, three techniques, Bayesian networks, Lo...",Obesity is a significant problem in population...,"[1, 1, 1, 0, 0, 0, 1, 1, 0]"


In [3]:
results = []

# sentence-bert


In [4]:
model = SentenceTransformer(
    "sbert"
#     'all-MiniLM-L6-v2', 
#     cache_folder = "../assets"
#     "../cache/huggingface/transformers/"
#     cache_folder = "../cache/huggingface/transformers"
)

In [5]:
sbert_queries = [model.encode(t) for t in tqdm(data["query"])]

  0%|          | 0/23 [00:00<?, ?it/s]

In [6]:
sbert_sents = [model.encode(t) for t in tqdm(data.text)]

  0%|          | 0/23 [00:00<?, ?it/s]

In [7]:
sbert_features = [
    np.concatenate([np.tile(q, (l.shape[0], 1)), l], axis = 1)
    for q, l in zip(sbert_queries, sbert_sents)
]

len(sbert_features), sbert_features[0].shape

(23, (15, 768))

In [8]:
sbert_features = [torch.tensor(x).cuda() for x in sbert_features]

### cosine similarity

In [9]:
sbert_cosine_scores = [s @ q.T for s, q in zip(tqdm(sbert_sents), sbert_queries)]

  0%|          | 0/23 [00:00<?, ?it/s]

In [10]:
sbert_cosine_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), sbert_cosine_scores)]
len(sbert_cosine_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [11]:
sbert_cosine_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), sbert_cosine_scores)]
len(sbert_cosine_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [12]:
results.append(
    dict(
        model = "sbert_cosine",
        ap_mean = np.mean(sbert_cosine_ap), 
        ap_std = 2 * np.std(sbert_cosine_ap)/np.sqrt(len(sbert_cosine_ap)),
        ra_mean = np.mean(sbert_cosine_ra), 
        ra_std = 2 * np.std(sbert_cosine_ra)/np.sqrt(len(sbert_cosine_ra))
    )
)
results

[{'model': 'sbert_cosine',
  'ap_mean': 0.5257125870061516,
  'ap_std': 0.09476285761725348,
  'ra_mean': 0.42043224651520683,
  'ra_std': 0.12152907549366028}]

### mlp

In [13]:
mlp_sbert = malnis.models.MLP(
    hidden_size = 300,
#     attention_heads = 4,
    num_layers = 3,
#     d_model = 1536
)
mlp_sbert.load_state_dict(
    torch.load("/home/jarobyte/scratch/malnis_dataset/mlp_pytorch/sbert/models/14628727_2.pt")
)
mlp_sbert.cuda()

MLP Classfier
Parameters: 501,901


MLP(
  (input_layer): Linear(in_features=768, out_features=300, bias=True)
  (layers): ModuleList(
    (0): Linear(in_features=300, out_features=300, bias=True)
    (1): Linear(in_features=300, out_features=300, bias=True)
    (2): Linear(in_features=300, out_features=300, bias=True)
  )
  (output_layer): Linear(in_features=300, out_features=1, bias=True)
)

In [14]:
mlp_sbert_scores = [mlp_sbert.predict(x) for x in sbert_features]
len(mlp_sbert_scores), mlp_sbert_scores[0].shape

(23, (15,))

In [15]:
mlp_sbert_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), mlp_sbert_scores)]
len(mlp_sbert_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [16]:
mlp_sbert_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), mlp_sbert_scores)]
len(mlp_sbert_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [17]:
results.append(
    dict(
        model = "mlp_sbert",
        ap_mean = np.mean(mlp_sbert_ap), 
        ap_std = 2 * np.std(mlp_sbert_ap)/np.sqrt(len(mlp_sbert_ap)),
        ra_mean = np.mean(mlp_sbert_ra), 
        ra_std = 2 * np.std(mlp_sbert_ra)/np.sqrt(len(mlp_sbert_ra))
    )
)
results

[{'model': 'sbert_cosine',
  'ap_mean': 0.5257125870061516,
  'ap_std': 0.09476285761725348,
  'ra_mean': 0.42043224651520683,
  'ra_std': 0.12152907549366028},
 {'model': 'mlp_sbert',
  'ap_mean': 0.5998024056534895,
  'ap_std': 0.12209525351711967,
  'ra_mean': 0.5400809277927244,
  'ra_std': 0.13652664562175304}]

### lstm

In [18]:
lstm_sbert = malnis.models.LSTM(
#     input_size = 1536,
    hidden_size = 500,
    num_layers = 3
)
lstm_sbert.load_state_dict(torch.load("/home/jarobyte/scratch/malnis_dataset/lstm/sbert/models/13932670_9.pt"))
lstm_sbert.cuda()

LSTM Classfier
Parameters: 17,097,001


LSTM(
  (lstm): LSTM(768, 500, num_layers=3, bidirectional=True)
  (out_layer): Linear(in_features=1000, out_features=1, bias=True)
)

In [19]:
lstm_sbert_scores = [lstm_sbert.predict(x) for x in sbert_features]
len(lstm_sbert_scores), lstm_sbert_scores[0].shape

(23, (15,))

In [20]:
lstm_sbert_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), lstm_sbert_scores)]
len(lstm_sbert_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [21]:
lstm_sbert_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), lstm_sbert_scores)]
len(lstm_sbert_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [22]:
results.append(
    dict(
        model = "lstm_sbert",
        ap_mean = np.mean(lstm_sbert_ap), 
        ap_std = 2 * np.std(lstm_sbert_ap)/np.sqrt(len(lstm_sbert_ap)),
        ra_mean = np.mean(lstm_sbert_ra), 
        ra_std = 2 * np.std(lstm_sbert_ra)/np.sqrt(len(lstm_sbert_ra))
    )
)
results

[{'model': 'sbert_cosine',
  'ap_mean': 0.5257125870061516,
  'ap_std': 0.09476285761725348,
  'ra_mean': 0.42043224651520683,
  'ra_std': 0.12152907549366028},
 {'model': 'mlp_sbert',
  'ap_mean': 0.5998024056534895,
  'ap_std': 0.12209525351711967,
  'ra_mean': 0.5400809277927244,
  'ra_std': 0.13652664562175304},
 {'model': 'lstm_sbert',
  'ap_mean': 0.6000311710746817,
  'ap_std': 0.11428589417682086,
  'ra_mean': 0.6308190572836513,
  'ra_std': 0.09576566024337917}]

### transformer

In [23]:
transformer_sbert = malnis.models.Transformer(
    dim_feedforward = 250,
    attention_heads = 4,
    num_layers = 3
)
transformer_sbert.load_state_dict(
    torch.load("/home/jarobyte/scratch/malnis_dataset/transformer/sbert/models/13932680_10.pt")
)
transformer_sbert.cuda()

Transformer Classfier
Parameters: 11,395,817


Transformer(
  (positional_embeddings): Embedding(512, 768)
  (transformer_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
    )
    (linear1): Linear(in_features=768, out_features=250, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=250, out_features=768, bias=True)
    (norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
        )
        (linear1): Linear(in_features=768, out_features=250, bias=True)
        (dropout): Dropo

In [24]:
transformer_sbert_scores = [transformer_sbert.predict(x.unsqueeze(0)) for x in sbert_features]
len(transformer_sbert_scores), transformer_sbert_scores[0].shape

(23, (15,))

In [25]:
transformer_sbert_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), transformer_sbert_scores)]
len(transformer_sbert_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [26]:
transformer_sbert_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), transformer_sbert_scores)]
len(transformer_sbert_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [27]:
results.append(
    dict(
        model = "transformer_sbert",
        ap_mean = np.mean(transformer_sbert_ap), 
        ap_std = 2 * np.std(transformer_sbert_ap)/np.sqrt(len(transformer_sbert_ap)),
        ra_mean = np.mean(transformer_sbert_ra), 
        ra_std = 2 * np.std(transformer_sbert_ra)/np.sqrt(len(transformer_sbert_ra))
    )
)
results

[{'model': 'sbert_cosine',
  'ap_mean': 0.5257125870061516,
  'ap_std': 0.09476285761725348,
  'ra_mean': 0.42043224651520683,
  'ra_std': 0.12152907549366028},
 {'model': 'mlp_sbert',
  'ap_mean': 0.5998024056534895,
  'ap_std': 0.12209525351711967,
  'ra_mean': 0.5400809277927244,
  'ra_std': 0.13652664562175304},
 {'model': 'lstm_sbert',
  'ap_mean': 0.6000311710746817,
  'ap_std': 0.11428589417682086,
  'ra_mean': 0.6308190572836513,
  'ra_std': 0.09576566024337917},
 {'model': 'transformer_sbert',
  'ap_mean': 0.5628612696690114,
  'ap_std': 0.11371952729894613,
  'ra_mean': 0.5100305207722087,
  'ra_std': 0.14347869983106168}]

## specter

In [28]:
model = SentenceTransformer(
    "specter"
#     'all-MiniLM-L6-v2', 
#     cache_folder = "../assets"
#     "../cache/huggingface/transformers/"
#     cache_folder = "../cache/huggingface/transformers"
)

In [29]:
specter_queries = [model.encode(t) for t in tqdm(data["query"])]

  0%|          | 0/23 [00:00<?, ?it/s]

In [30]:
specter_sents = [model.encode(t) for t in tqdm(data.text)]

  0%|          | 0/23 [00:00<?, ?it/s]

In [31]:
specter_features = [
    np.concatenate([np.tile(q, (l.shape[0], 1)), l], axis = 1)
    for q, l in zip(specter_queries, specter_sents)
]

len(specter_features), specter_features[0].shape

(23, (15, 1536))

In [32]:
specter_features = [torch.tensor(x).cuda() for x in specter_features]

### euclidean distance

In [33]:
specter_euclidean_scores = [2 / (1 + np.exp((s-q)**2).sum(1)) for s, q in zip(tqdm(specter_sents), specter_queries)]

  0%|          | 0/23 [00:00<?, ?it/s]

In [34]:
specter_euclidean_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), specter_euclidean_scores)]
len(specter_euclidean_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [35]:
specter_euclidean_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), specter_euclidean_scores)]
len(specter_euclidean_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [36]:
results.append(
    dict(
        model = "specter_euclidean",
        ap_mean = np.mean(specter_euclidean_ap), 
        ap_std = 2 * np.std(specter_euclidean_ap)/np.sqrt(len(specter_euclidean_ap)),
        ra_mean = np.mean(specter_euclidean_ra), 
        ra_std = 2 * np.std(specter_euclidean_ra)/np.sqrt(len(specter_euclidean_ra))
    )
)
results

[{'model': 'sbert_cosine',
  'ap_mean': 0.5257125870061516,
  'ap_std': 0.09476285761725348,
  'ra_mean': 0.42043224651520683,
  'ra_std': 0.12152907549366028},
 {'model': 'mlp_sbert',
  'ap_mean': 0.5998024056534895,
  'ap_std': 0.12209525351711967,
  'ra_mean': 0.5400809277927244,
  'ra_std': 0.13652664562175304},
 {'model': 'lstm_sbert',
  'ap_mean': 0.6000311710746817,
  'ap_std': 0.11428589417682086,
  'ra_mean': 0.6308190572836513,
  'ra_std': 0.09576566024337917},
 {'model': 'transformer_sbert',
  'ap_mean': 0.5628612696690114,
  'ap_std': 0.11371952729894613,
  'ra_mean': 0.5100305207722087,
  'ra_std': 0.14347869983106168},
 {'model': 'specter_euclidean',
  'ap_mean': 0.531631446041713,
  'ap_std': 0.11548072630416334,
  'ra_mean': 0.5047756174274717,
  'ra_std': 0.12595320382763603}]

### mlp

In [37]:
mlp_specter = malnis.models.MLP(
    hidden_size = 450,
#     attention_heads = 4,
    num_layers = 4,
    input_size = 1536
)
mlp_specter.load_state_dict(
    torch.load("/home/jarobyte/scratch/malnis_dataset/mlp_pytorch/specter/models/14678619_2.pt")
)
mlp_specter.cuda()

MLP Classfier
Parameters: 1,503,901


MLP(
  (input_layer): Linear(in_features=1536, out_features=450, bias=True)
  (layers): ModuleList(
    (0): Linear(in_features=450, out_features=450, bias=True)
    (1): Linear(in_features=450, out_features=450, bias=True)
    (2): Linear(in_features=450, out_features=450, bias=True)
    (3): Linear(in_features=450, out_features=450, bias=True)
  )
  (output_layer): Linear(in_features=450, out_features=1, bias=True)
)

In [38]:
mlp_specter_scores = [mlp_specter.predict(x) for x in specter_features]
len(mlp_specter_scores), mlp_specter_scores[0].shape

(23, (15,))

In [39]:
mlp_specter_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), mlp_specter_scores)]
len(mlp_specter_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [40]:
mlp_specter_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), mlp_specter_scores)]
len(mlp_specter_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [41]:
results.append(
    dict(
        model = "mlp_specter",
        ap_mean = np.mean(mlp_specter_ap), 
        ap_std = 2 * np.std(mlp_specter_ap)/np.sqrt(len(mlp_specter_ap)),
        ra_mean = np.mean(mlp_specter_ra), 
        ra_std = 2 * np.std(mlp_specter_ra)/np.sqrt(len(mlp_specter_ra))
    )
)
results

[{'model': 'sbert_cosine',
  'ap_mean': 0.5257125870061516,
  'ap_std': 0.09476285761725348,
  'ra_mean': 0.42043224651520683,
  'ra_std': 0.12152907549366028},
 {'model': 'mlp_sbert',
  'ap_mean': 0.5998024056534895,
  'ap_std': 0.12209525351711967,
  'ra_mean': 0.5400809277927244,
  'ra_std': 0.13652664562175304},
 {'model': 'lstm_sbert',
  'ap_mean': 0.6000311710746817,
  'ap_std': 0.11428589417682086,
  'ra_mean': 0.6308190572836513,
  'ra_std': 0.09576566024337917},
 {'model': 'transformer_sbert',
  'ap_mean': 0.5628612696690114,
  'ap_std': 0.11371952729894613,
  'ra_mean': 0.5100305207722087,
  'ra_std': 0.14347869983106168},
 {'model': 'specter_euclidean',
  'ap_mean': 0.531631446041713,
  'ap_std': 0.11548072630416334,
  'ra_mean': 0.5047756174274717,
  'ra_std': 0.12595320382763603},
 {'model': 'mlp_specter',
  'ap_mean': 0.6519329492485555,
  'ap_std': 0.11413645829468397,
  'ra_mean': 0.6544953785203146,
  'ra_std': 0.11705000120405035}]

### lstm

In [42]:
lstm_specter = malnis.models.LSTM(
    input_size = 1536,
    hidden_size = 500,
    num_layers = 1
)
lstm_specter.load_state_dict(torch.load("/home/jarobyte/scratch/malnis_dataset/lstm/specter/models/13925873_9.pt"))
lstm_specter.cuda()

LSTM Classfier
Parameters: 8,153,001


LSTM(
  (lstm): LSTM(1536, 500, bidirectional=True)
  (out_layer): Linear(in_features=1000, out_features=1, bias=True)
)

In [43]:
lstm_specter_scores = [lstm_specter.predict(x) for x in specter_features]
len(lstm_specter_scores), lstm_specter_scores[0].shape

(23, (15,))

In [44]:
lstm_specter_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), lstm_specter_scores)]
len(lstm_specter_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [45]:
lstm_specter_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), lstm_specter_scores)]
len(lstm_specter_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [46]:
results.append(
    dict(
        model = "lstm_specter",
        ap_mean = np.mean(lstm_specter_ap), 
        ap_std = 2 * np.std(lstm_specter_ap)/np.sqrt(len(lstm_specter_ap)),
        ra_mean = np.mean(lstm_specter_ra), 
        ra_std = 2 * np.std(lstm_specter_ra)/np.sqrt(len(lstm_specter_ra))
    )
)
results

[{'model': 'sbert_cosine',
  'ap_mean': 0.5257125870061516,
  'ap_std': 0.09476285761725348,
  'ra_mean': 0.42043224651520683,
  'ra_std': 0.12152907549366028},
 {'model': 'mlp_sbert',
  'ap_mean': 0.5998024056534895,
  'ap_std': 0.12209525351711967,
  'ra_mean': 0.5400809277927244,
  'ra_std': 0.13652664562175304},
 {'model': 'lstm_sbert',
  'ap_mean': 0.6000311710746817,
  'ap_std': 0.11428589417682086,
  'ra_mean': 0.6308190572836513,
  'ra_std': 0.09576566024337917},
 {'model': 'transformer_sbert',
  'ap_mean': 0.5628612696690114,
  'ap_std': 0.11371952729894613,
  'ra_mean': 0.5100305207722087,
  'ra_std': 0.14347869983106168},
 {'model': 'specter_euclidean',
  'ap_mean': 0.531631446041713,
  'ap_std': 0.11548072630416334,
  'ra_mean': 0.5047756174274717,
  'ra_std': 0.12595320382763603},
 {'model': 'mlp_specter',
  'ap_mean': 0.6519329492485555,
  'ap_std': 0.11413645829468397,
  'ra_mean': 0.6544953785203146,
  'ra_std': 0.11705000120405035},
 {'model': 'lstm_specter',
  'ap_mea

### transformer

In [47]:
transformer_specter = malnis.models.Transformer(
    dim_feedforward = 350,
    attention_heads = 4,
    num_layers = 3,
    d_model = 1536
)
transformer_specter.load_state_dict(
    torch.load("/home/jarobyte/scratch/malnis_dataset/transformer/specter/models/14158104_4.pt")
)
transformer_specter.cuda()

Transformer Classfier
Parameters: 42,894,201


Transformer(
  (positional_embeddings): Embedding(512, 1536)
  (transformer_layer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=1536, out_features=1536, bias=True)
    )
    (linear1): Linear(in_features=1536, out_features=350, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=350, out_features=1536, bias=True)
    (norm1): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((1536,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (encoder): TransformerEncoder(
    (layers): ModuleList(
      (0): TransformerEncoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=1536, out_features=1536, bias=True)
        )
        (linear1): Linear(in_features=1536, out_features=350, bias=True)
        (dropo

In [48]:
transformer_specter_scores = [transformer_specter.predict(x.unsqueeze(0)) for x in specter_features]
len(transformer_specter_scores), transformer_specter_scores[0].shape

(23, (15,))

In [49]:
transformer_specter_ap = [average_precision_score(l, s) for l, s in zip(tqdm(data.relevance), transformer_specter_scores)]
len(transformer_specter_ap)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [50]:
transformer_specter_ra = [roc_auc_score(l, s) for l, s in zip(tqdm(data.relevance), transformer_specter_scores)]
len(transformer_specter_ra)

  0%|          | 0/23 [00:00<?, ?it/s]

23

In [51]:
results.append(
    dict(
        model = "transformer_specter",
        ap_mean = np.mean(transformer_specter_ap), 
        ap_std = 2 * np.std(transformer_specter_ap)/np.sqrt(len(transformer_specter_ap)),
        ra_mean = np.mean(transformer_specter_ra), 
        ra_std = 2 * np.std(transformer_specter_ra)/np.sqrt(len(transformer_specter_ra))
    )
)
results

[{'model': 'sbert_cosine',
  'ap_mean': 0.5257125870061516,
  'ap_std': 0.09476285761725348,
  'ra_mean': 0.42043224651520683,
  'ra_std': 0.12152907549366028},
 {'model': 'mlp_sbert',
  'ap_mean': 0.5998024056534895,
  'ap_std': 0.12209525351711967,
  'ra_mean': 0.5400809277927244,
  'ra_std': 0.13652664562175304},
 {'model': 'lstm_sbert',
  'ap_mean': 0.6000311710746817,
  'ap_std': 0.11428589417682086,
  'ra_mean': 0.6308190572836513,
  'ra_std': 0.09576566024337917},
 {'model': 'transformer_sbert',
  'ap_mean': 0.5628612696690114,
  'ap_std': 0.11371952729894613,
  'ra_mean': 0.5100305207722087,
  'ra_std': 0.14347869983106168},
 {'model': 'specter_euclidean',
  'ap_mean': 0.531631446041713,
  'ap_std': 0.11548072630416334,
  'ra_mean': 0.5047756174274717,
  'ra_std': 0.12595320382763603},
 {'model': 'mlp_specter',
  'ap_mean': 0.6519329492485555,
  'ap_std': 0.11413645829468397,
  'ra_mean': 0.6544953785203146,
  'ra_std': 0.11705000120405035},
 {'model': 'lstm_specter',
  'ap_mea

In [52]:
results_df = pd.DataFrame(results)
show(results_df)

(8, 5)


,model,ap_mean,ap_std,ra_mean,ra_std
0,sbert_cosine,0.525713,0.094763,0.420432,0.121529
1,mlp_sbert,0.599802,0.122095,0.540081,0.136527
2,lstm_sbert,0.600031,0.114286,0.630819,0.095766
3,transformer_sbert,0.562861,0.113720,0.510031,0.143479
4,specter_euclidean,0.531631,0.115481,0.504776,0.125953


In [53]:
print(results_df.round(3).to_latex())

\begin{tabular}{llrrrr}
\toprule
{} &                model &  ap\_mean &  ap\_std &  ra\_mean &  ra\_std \\
\midrule
0 &         sbert\_cosine &    0.526 &   0.095 &    0.420 &   0.122 \\
1 &            mlp\_sbert &    0.600 &   0.122 &    0.540 &   0.137 \\
2 &           lstm\_sbert &    0.600 &   0.114 &    0.631 &   0.096 \\
3 &    transformer\_sbert &    0.563 &   0.114 &    0.510 &   0.143 \\
4 &    specter\_euclidean &    0.532 &   0.115 &    0.505 &   0.126 \\
5 &          mlp\_specter &    0.652 &   0.114 &    0.654 &   0.117 \\
6 &         lstm\_specter &    0.637 &   0.107 &    0.627 &   0.109 \\
7 &  transformer\_specter &    0.623 &   0.104 &    0.590 &   0.118 \\
\bottomrule
\end{tabular}



/tmp/ipykernel_3702160/2842510611.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(results_df.round(3).to_latex())


In [54]:
results_df.to_pickle("quotes_results_deep.pkl")